# Assessing fire risk by location in NYC

## Background
Using data provided by NYC OpenData, this notebook walks through the steps of analyzing fire risk in New York City.

## Import Libraries

In [1]:
# Data analysis and visualization
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Interactive maps
import folium
from folium.plugins import HeatMap

# Machine Learning
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, roc_auc_score, auc
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.pipeline import Pipeline

## Load and describe data

Note: Data was filtered on the NYC OpenData site to only include incident classification groups that were fire-related (Structural and NonStructural Fires) prior to export.

In [9]:
alarms_data = pd.read_csv('../data/In-Service_Alarm_Box_Locations.csv')
fires_data = pd.read_csv('../data/Fire_Incident_Dispatch_Data.csv')

In [20]:
alarms_data

,BOROBOX,BOX_TYPE,LOCATION,ZIP,BOROUGH,COMMUNITYDISTICT,CITYCOUNCIL,LATITUDE,LONGITUDE,Location Point
0,B2653,ERS,3 AVE & 65 ST,11220.0,Brooklyn,BK07,38.0,40.639320,-74.023549,POINT (-74.02354939 40.63932033)
1,Q7917,BARS,WOODSIDE AVE & 69 ST,11377.0,Queens,QN02,26.0,40.742686,-73.895652,POINT (-73.89565167 40.7426855)
2,B0801,ERS,MYRTLE AVE & PALMETTO ST,11237.0,Brooklyn,QN05,34.0,40.699532,-73.911035,POINT (-73.9110349 40.69953211)
3,B1046,ERS,NEW YORK AVE & LEFFERTS AVE,11225.0,Brooklyn,BK09,40.0,40.662534,-73.947914,POINT (-73.94791393 40.66253364)
4,B0109,ERS,RIVER & NORTH 3 STS,11211.0,Brooklyn,BK01,33.0,40.718376,-73.964621,POINT (-73.96462115 40.71837562)
...,...,...,...,...,...,...,...,...,...,...
13003,B1169,ERS,3 AVE & 86 ST,11209.0,Brooklyn,BK10,43.0,40.623463,-74.031060,POINT (-74.03105992 40.6234629)
13004,Q6551,BARS,GCP & 165 ST,11432.0,Queens,QN08,24.0,40.716787,-73.800992,POINT (-73.80099165 40.71678736)
13005,Q6446,BARS,LITTLE NECK PWAY 500' N OF CULLMAN AVE,11362.0,Queens,QN11,23.0,40.756611,-73.722812,POINT (-73.72281201 40.75661118)
13006,B3874,ERS,AVENUE T & W 13 ST,11223.0,Brooklyn,BK11,47.0,40.598038,-73.984494,POINT (-73.98449383 40.59803845)


In [21]:
fires_data

,STARFIRE_INCIDENT_ID,INCIDENT_DATETIME,ALARM_BOX_BOROUGH,ALARM_BOX_NUMBER,ALARM_BOX_LOCATION,INCIDENT_BOROUGH,ZIPCODE,POLICEPRECINCT,CITYCOUNCILDISTRICT,COMMUNITYDISTRICT,...,FIRST_ACTIVATION_DATETIME,FIRST_ON_SCENE_DATETIME,INCIDENT_CLOSE_DATETIME,VALID_DISPATCH_RSPNS_TIME_INDC,VALID_INCIDENT_RSPNS_TIME_INDC,INCIDENT_RESPONSE_SECONDS_QY,INCIDENT_TRAVEL_TM_SECONDS_QY,ENGINES_ASSIGNED_QUANTITY,LADDERS_ASSIGNED_QUANTITY,OTHER_UNITS_ASSIGNED_QUANTITY
0,1800111510150000,01/01/2018 12:02:05 AM,QUEENS,1151,BAY 28 ST & SUNNYSIDE ST,QUEENS,11691.0,101.0,31.0,414.0,...,01/01/2018 12:03:41 AM,01/01/2018 12:07:55 AM,01/01/2018 12:15:08 AM,N,Y,350,280,1,1,0
1,1800197070150000,01/01/2018 12:02:54 AM,QUEENS,9707,75 AVE & 178 ST,QUEENS,11366.0,107.0,24.0,408.0,...,01/01/2018 12:03:58 AM,01/01/2018 12:07:15 AM,01/01/2018 12:10:32 AM,N,Y,261,204,1,1,0
2,1800137760240000,01/01/2018 12:03:53 AM,BROOKLYN,3776,CHURCH AVE & STORY ST,BROOKLYN,11218.0,66.0,39.0,312.0,...,01/01/2018 12:04:56 AM,01/01/2018 12:08:00 AM,01/01/2018 12:10:40 AM,N,Y,247,205,1,0,0
3,1800152100150000,01/01/2018 12:13:46 AM,QUEENS,5210,LINDEN BLVD & 121 ST,QUEENS,11420.0,106.0,28.0,410.0,...,01/01/2018 12:14:25 AM,01/01/2018 12:16:26 AM,01/01/2018 12:21:10 AM,N,Y,160,141,3,2,1
4,1800131730120010,01/01/2018 12:14:20 AM,BRONX,3173,CROTONA AVE & 182 ST,BRONX,10457.0,48.0,15.0,206.0,...,01/01/2018 12:14:59 AM,01/01/2018 12:17:14 AM,01/01/2018 12:26:54 AM,N,Y,174,160,3,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134200,2112560000000000,05/05/2021 09:28:00 PM,QUEENS,6117,CORBETT RD & 221 ST,QUEENS,11361.0,111.0,19.0,411.0,...,05/05/2021 09:29:00 PM,05/05/2021 09:34:00 PM,05/05/2021 09:41:00 PM,N,Y,319,264,1,1,0
134201,2112510000000000,05/05/2021 09:31:00 PM,MANHATTAN,1442,LEXINGTON AVE & 121 ST,MANHATTAN,10035.0,25.0,9.0,111.0,...,05/05/2021 09:31:00 PM,05/05/2021 09:33:00 PM,05/05/2021 10:48:00 PM,N,Y,151,126,4,2,1
134202,2112510000000000,05/05/2021 10:23:00 PM,MANHATTAN,1325,AMSTERDAM AVE & 109 ST,MANHATTAN,10025.0,24.0,7.0,107.0,...,05/05/2021 10:24:00 PM,05/05/2021 10:27:00 PM,05/05/2021 10:41:00 PM,N,Y,210,184,3,2,1
134203,2112530000000000,05/05/2021 11:16:00 PM,BRONX,3170,BRYANT AVE & BRONX PARK SO.,BRONX,10460.0,48.0,15.0,206.0,...,05/05/2021 11:17:00 PM,05/05/2021 11:21:00 PM,05/06/2021 12:01:00 AM,N,Y,267,233,3,2,1


In [19]:

merged = pd.merge(left=alarms_data, right=fires_data, left_on='LOCATION', right_on='ALARM_BOX_LOCATION')
merged = merged[['BOROBOX',
                 'LOCATION',
                 'BOROUGH',
                 'ALARM_BOX_NUMBER',
                 'ALARM_BOX_LOCATION',
                 'ALARM_BOX_BOROUGH',
               ]]
merged


,BOROBOX,LOCATION,BOROUGH,ALARM_BOX_NUMBER,ALARM_BOX_LOCATION,ALARM_BOX_BOROUGH
0,B2653,3 AVE & 65 ST,Brooklyn,2653,3 AVE & 65 ST,BROOKLYN
1,B2653,3 AVE & 65 ST,Brooklyn,2653,3 AVE & 65 ST,BROOKLYN
2,B2653,3 AVE & 65 ST,Brooklyn,2653,3 AVE & 65 ST,BROOKLYN
3,B2653,3 AVE & 65 ST,Brooklyn,2653,3 AVE & 65 ST,BROOKLYN
4,B2653,3 AVE & 65 ST,Brooklyn,2653,3 AVE & 65 ST,BROOKLYN
...,...,...,...,...,...,...
62106,Q4713,MERRICK BLVD & BAISLEY BLVD,Queens,4713,MERRICK BLVD & BAISLEY BLVD,QUEENS
62107,Q4713,MERRICK BLVD & BAISLEY BLVD,Queens,4713,MERRICK BLVD & BAISLEY BLVD,QUEENS
62108,Q4713,MERRICK BLVD & BAISLEY BLVD,Queens,4713,MERRICK BLVD & BAISLEY BLVD,QUEENS
62109,Q4713,MERRICK BLVD & BAISLEY BLVD,Queens,4713,MERRICK BLVD & BAISLEY BLVD,QUEENS


In [3]:
# Remove unnecessary columns
data = raw_data[['INCIDENT_DATETIME',
                 'ALARM_BOX_BOROUGH',
                 'ALARM_BOX_NUMBER',
                 'ALARM_BOX_LOCATION',
                 'INCIDENT_BOROUGH',
                 'ZIPCODE',
                 'INCIDENT_CLASSIFICATION',
                 'INCIDENT_CLASSIFICATION_GROUP',
                 'DISPATCH_RESPONSE_SECONDS_QY',
                 'INCIDENT_RESPONSE_SECONDS_QY',
                 'INCIDENT_TRAVEL_TM_SECONDS_QY',
                 'ENGINES_ASSIGNED_QUANTITY',
                 'LADDERS_ASSIGNED_QUANTITY',
                 'OTHER_UNITS_ASSIGNED_QUANTITY',]]
data.head()

,INCIDENT_DATETIME,ALARM_BOX_BOROUGH,ALARM_BOX_NUMBER,ALARM_BOX_LOCATION,INCIDENT_BOROUGH,ZIPCODE,INCIDENT_CLASSIFICATION,INCIDENT_CLASSIFICATION_GROUP,DISPATCH_RESPONSE_SECONDS_QY,INCIDENT_RESPONSE_SECONDS_QY,INCIDENT_TRAVEL_TM_SECONDS_QY,ENGINES_ASSIGNED_QUANTITY,LADDERS_ASSIGNED_QUANTITY,OTHER_UNITS_ASSIGNED_QUANTITY
0,01/01/2018 12:02:05 AM,QUEENS,1151,BAY 28 ST & SUNNYSIDE ST,QUEENS,11691.0,Private Dwelling Fire,Structural Fires,70,350,280,1,1,0
1,01/01/2018 12:02:54 AM,QUEENS,9707,75 AVE & 178 ST,QUEENS,11366.0,Private Dwelling Fire,Structural Fires,57,261,204,1,1,0
2,01/01/2018 12:03:53 AM,BROOKLYN,3776,CHURCH AVE & STORY ST,BROOKLYN,11218.0,Demolition Debris or Rubbish Fire,NonStructural Fires,42,247,205,1,0,0
3,01/01/2018 12:13:46 AM,QUEENS,5210,LINDEN BLVD & 121 ST,QUEENS,11420.0,Demolition Debris or Rubbish Fire,NonStructural Fires,19,160,141,3,2,1
4,01/01/2018 12:14:20 AM,BRONX,3173,CROTONA AVE & 182 ST,BRONX,10457.0,Multiple Dwelling 'A' - Food on the stove fire,Structural Fires,14,174,160,3,2,1


In [5]:
data.groupby('INCIDENT_CLASSIFICATION_GROUP').count()

,INCIDENT_DATETIME,ALARM_BOX_BOROUGH,ALARM_BOX_NUMBER,ALARM_BOX_LOCATION,INCIDENT_BOROUGH,ZIPCODE,INCIDENT_CLASSIFICATION,DISPATCH_RESPONSE_SECONDS_QY,INCIDENT_RESPONSE_SECONDS_QY,INCIDENT_TRAVEL_TM_SECONDS_QY,ENGINES_ASSIGNED_QUANTITY,LADDERS_ASSIGNED_QUANTITY,OTHER_UNITS_ASSIGNED_QUANTITY
INCIDENT_CLASSIFICATION_GROUP,,,,,,,,,,,,,
NonStructural Fires,47385,47385,47385,47378,47385,41827,47385,47385,47385,47385,47385,47385,47385
Structural Fires,86820,86820,86820,86817,86820,86282,86820,86820,86820,86820,86820,86820,86820


In [8]:
building_fires = data[data['INCIDENT_CLASSIFICATION_GROUP']=='Structural Fires']
building_fires

,INCIDENT_DATETIME,ALARM_BOX_BOROUGH,ALARM_BOX_NUMBER,ALARM_BOX_LOCATION,INCIDENT_BOROUGH,ZIPCODE,INCIDENT_CLASSIFICATION,INCIDENT_CLASSIFICATION_GROUP,DISPATCH_RESPONSE_SECONDS_QY,INCIDENT_RESPONSE_SECONDS_QY,INCIDENT_TRAVEL_TM_SECONDS_QY,ENGINES_ASSIGNED_QUANTITY,LADDERS_ASSIGNED_QUANTITY,OTHER_UNITS_ASSIGNED_QUANTITY
0,01/01/2018 12:02:05 AM,QUEENS,1151,BAY 28 ST & SUNNYSIDE ST,QUEENS,11691.0,Private Dwelling Fire,Structural Fires,70,350,280,1,1,0
1,01/01/2018 12:02:54 AM,QUEENS,9707,75 AVE & 178 ST,QUEENS,11366.0,Private Dwelling Fire,Structural Fires,57,261,204,1,1,0
4,01/01/2018 12:14:20 AM,BRONX,3173,CROTONA AVE & 182 ST,BRONX,10457.0,Multiple Dwelling 'A' - Food on the stove fire,Structural Fires,14,174,160,3,2,1
5,01/01/2018 12:15:06 AM,QUEENS,4622,41 AVE & 169 ST,QUEENS,11358.0,Private Dwelling Fire,Structural Fires,98,392,294,1,1,0
9,01/01/2018 12:26:18 AM,BROOKLYN,356,PARK AVE & MARCUS GARVEY BLVD/SUMNER AVE,BROOKLYN,11206.0,Multiple Dwelling 'A' - Food on the stove fire,Structural Fires,39,220,181,3,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134200,05/05/2021 09:28:00 PM,QUEENS,6117,CORBETT RD & 221 ST,QUEENS,11361.0,Private Dwelling Fire,Structural Fires,55,319,264,1,1,0
134201,05/05/2021 09:31:00 PM,MANHATTAN,1442,LEXINGTON AVE & 121 ST,MANHATTAN,10035.0,Multiple Dwelling 'A' - Other fire,Structural Fires,25,151,126,4,2,1
134202,05/05/2021 10:23:00 PM,MANHATTAN,1325,AMSTERDAM AVE & 109 ST,MANHATTAN,10025.0,Multiple Dwelling 'A' - Food on the stove fire,Structural Fires,26,210,184,3,2,1
134203,05/05/2021 11:16:00 PM,BRONX,3170,BRYANT AVE & BRONX PARK SO.,BRONX,10460.0,Multiple Dwelling 'A' - Other fire,Structural Fires,34,267,233,3,2,1


In [ ]:
fire_map = folium.Map(location=[40.73, -73.94], zoom_start=14, tiles='Stamen Terrain')
